<a href="https://colab.research.google.com/github/sncy/No-way/blob/master/Trading_Pattern_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Developing Strategies

##Three Green Soldiers Gold

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/sample.csv')
columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD15min3Oct2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD30min1Sept2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD1hour10July2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD4hour8July2022.csv', names=columns, header=None)
df

,Date,Time,Open,High,Low,Close,Volume
0,2023.10.03,01:30,1828.62,1828.67,1827.56,1827.72,537
1,2023.10.03,01:45,1827.74,1828.79,1827.44,1828.21,372
2,2023.10.03,02:00,1828.20,1828.20,1822.33,1822.98,792
3,2023.10.03,02:15,1822.95,1823.13,1820.44,1822.36,1069
4,2023.10.03,02:30,1822.36,1824.56,1822.04,1824.43,713


In [ ]:
# Define the candle color
df['Candle Color'] = np.where(df['Open'] > df['Close'] + 0.05, 'red', np.where(df['Open'] < df['Close'] - 0.05, 'green', 'doji'))

# Find the 'Three Green Soldiers' pattern
df['Three Green Soldiers'] = np.where((df['Candle Color'] == 'green') &
                                      (df['Candle Color'].shift(1) == 'green') &
                                      (df['Candle Color'].shift(2) == 'green'), 1, 0)

df_three_green_soldiers = df[df['Three Green Soldiers'] == 1]
df_three_green_soldiers

,Date,Time,Open,High,Low,Close,Volume,Candle Color,Three Green Soldiers
18,2023.10.03,06:00,1817.36,1818.52,1817.25,1818.36,591,green,1
22,2023.10.03,07:00,1820.49,1821.68,1820.49,1821.62,757,green,1
37,2023.10.03,10:45,1822.66,1824.14,1822.56,1824.01,969,green,1
38,2023.10.03,11:00,1824.03,1828.14,1824.03,1827.83,1298,green,1
39,2023.10.03,11:15,1827.83,1829.09,1827.22,1828.80,1137,green,1
...,...,...,...,...,...,...,...,...,...
3343,2023.11.22,09:15,2002.41,2003.39,2001.90,2002.54,1822,green,1
3344,2023.11.22,09:30,2002.60,2003.13,2001.80,2002.91,2144,green,1
3345,2023.11.22,09:45,2002.88,2004.00,2002.07,2003.71,2005,green,1
3346,2023.11.22,10:00,2003.66,2006.37,2003.27,2005.47,3347,green,1


In [ ]:
# Initialize the capital
capital = 10000

# Initialize the counters for winning trades, losing trades and cases where both levels are hit
winning_trades = 0
losing_trades = 0
both_levels_hit = 0

# Loop through the DataFrame
for i in range(2, len(df)):
    # Check if the 'Three Green Soldiers' pattern is detected
    if df['Three Green Soldiers'].iloc[i] == 1:
        trade_open = True
        entry_price = df['Close'].iloc[i]
        stop_loss = entry_price - 3
        take_profit = entry_price + 3
        j = i
        while trade_open == True:
          # Check if both levels are hit
          if df['Low'].iloc[j+1] < stop_loss and df['High'].iloc[j+1] > take_profit:
              both_levels_hit += 1
              trade_open = False
          else:
              # Assume that the stop loss level is hit if the low price of the next candle is below the stop loss level
              if df['Low'].iloc[j+1] < stop_loss:
                  # Decrease the capital by 1%
                  capital *= 0.99
                  losing_trades += 1
                  trade_open = False
              # Assume that the take profit level is hit if the high price of the next candle is above the take profit level
              elif df['High'].iloc[j+1] > take_profit:
                  # Increase the capital by 1%
                  capital *= 1.01
                  winning_trades += 1
                  trade_open = False
          j += 1

# Print the final capital and the number of winning and losing trades
total_trades = winning_trades + losing_trades
winning_rate = (winning_trades / total_trades) * 100

print(f"Final capital: {capital}")
print(f"Number of winning trades: {winning_trades}")
print(f"Number of losing trades: {losing_trades}")
print(f"Number of cases where both levels are hit: {both_levels_hit}")
print(f"Winning rate: {winning_rate:.2f}%")

Final capital: 11524.665004286391
Number of winning trades: 189
Number of losing trades: 173
Number of cases where both levels are hit: 2
Winning rate: 52.21%


##A Engulfing Wave (Gold 15)

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/sample.csv')
columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD15min3Oct2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD30min1Sept2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD1hour10July2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD4hour8July2022.csv', names=columns, header=None)
df

,Date,Time,Open,High,Low,Close,Volume
0,2023.10.03,01:30,1828.62,1828.67,1827.56,1827.72,537
1,2023.10.03,01:45,1827.74,1828.79,1827.44,1828.21,372
2,2023.10.03,02:00,1828.20,1828.20,1822.33,1822.98,792
3,2023.10.03,02:15,1822.95,1823.13,1820.44,1822.36,1069
4,2023.10.03,02:30,1822.36,1824.56,1822.04,1824.43,713
...,...,...,...,...,...,...,...
3371,2023.11.22,16:15,1999.85,2000.65,1998.85,1999.55,4369
3372,2023.11.22,16:30,1999.56,2002.13,1998.82,2001.97,5221
3373,2023.11.22,16:45,2001.99,2002.39,2000.90,2000.99,4154
3374,2023.11.22,17:00,2001.15,2001.50,1992.12,1993.30,6110


In [ ]:
# Add 'Candle Color' column
df['Candle Color'] = np.where(df['Open'] > df['Close'] + 0.05, 'red', np.where(df['Open'] < df['Close'] - 0.05, 'green', 'doji'))

# Add 'Candle Pattern' column
df['Candle Pattern'] = 'other'

# 参数调整
# DOJI 烛体的大小
doji_body_threshold = 0.05
# HAMMER 烛体的长度比例
hammer_body_ratio_from = 5/18
hammer_body_ratio_to = 7/18
# HAMMER 烛体方影线的长度比例
hammer_shadow_ratio = 1/14


# DOJI
# The absolute length of the real body is less or equal to doji_body_threshold
doji_condition = (df['Open'] - df['Close']).abs() <= doji_body_threshold


# HAMMER
# - The absolute length of the real body must be:
#   - Greater than or equal to hammer_body_ratio_from of the entire candle range.
#   - Less than or equal to hammer_body_ratio_to of the entire candle range.
# - Also either:
#   - The upper shadow is less than or equal to hammer_shadow_ratio of the candle range.
#   - OR the lower shadow is less than or equal to hammer_shadow_ratio of the candle range.
hammer_condition = (((df['Open'] - df['Close']).abs() >= (df['High'] - df['Low']) * hammer_body_ratio_from) & \
                   ((df['Open'] - df['Close']).abs() <= (df['High'] - df['Low']) * hammer_body_ratio_to) & \
                   (((df['High'] - df[['Open', 'Close']].max(axis=1)) <= (df['High'] - df['Low']) * hammer_shadow_ratio) | \
                   ((df[['Open', 'Close']].min(axis=1) - df['Low']) <= (df['High'] - df['Low']) * hammer_shadow_ratio)))


# ENGULFING
# - B-Engulfing: The current candle body high is higher and the body low is lower than the previous candle.
# - A-Engulfing: The current candle body high is higher and the body low is lower than the previous candle shadow.
b_engulfing_condition = (df[['Open', 'Close']].max(axis=1) >= df[['Open', 'Close']].max(axis=1).shift(1)) & \
                        (df[['Open', 'Close']].min(axis=1) <= df[['Open', 'Close']].min(axis=1).shift(1))
a_engulfing_condition = (df[['Open', 'Close']].max(axis=1) >= df['High'].shift(1)) & \
                        (df[['Open', 'Close']].min(axis=1) <= df['Low'].shift(1))


# Apply hammer condition
df.loc[doji_condition, 'Candle Pattern'] = 'doji'
df.loc[hammer_condition, 'Candle Pattern'] = 'hammer'
df.loc[b_engulfing_condition, 'Candle Pattern'] = 'B engulfing'
df.loc[a_engulfing_condition, 'Candle Pattern'] = 'A engulfing'

In [ ]:
pattern = df[df['Candle Pattern'] == 'A engulfing']
pattern

,Date,Time,Open,High,Low,Close,Volume,Candle Color,Candle Pattern
60,2023.10.03,16:30,1823.46,1831.30,1823.27,1830.39,1306,green,A engulfing
378,2023.10.09,04:00,1852.38,1852.72,1847.28,1847.57,1281,red,A engulfing
786,2023.10.13,14:00,1887.50,1891.99,1887.35,1891.25,3311,green,A engulfing
877,2023.10.16,13:45,1915.93,1918.35,1915.52,1917.98,1853,green,A engulfing
1142,2023.10.19,11:00,1949.21,1954.42,1949.10,1953.20,2991,green,A engulfing
2206,2023.11.06,01:00,1992.70,1992.71,1988.73,1988.93,964,red,A engulfing
2242,2023.11.06,10:00,1983.07,1986.13,1982.82,1985.70,2595,green,A engulfing
2767,2023.11.14,03:15,1948.15,1948.37,1946.19,1946.45,1181,red,A engulfing
3084,2023.11.17,13:30,1990.95,1991.04,1987.33,1987.87,2479,red,A engulfing
3232,2023.11.21,04:30,1988.75,1992.66,1988.75,1992.42,2204,green,A engulfing


In [ ]:
# define a new column 'A Engulfing Wave'
df['A Engulfing Wave'] = 0

# iterate over the DataFrame
for i in range(2, len(df)):
    # check if current 'Candle Pattern' is 'A Engulfing'
    if df.loc[i, 'Candle Pattern'] == 'A engulfing':
        # if green, then buy trade, look if got ABC pattern, B lowest must bigger than A lowest
        if df.loc[i, 'Candle Color'] == 'green' and (df.loc[i-1, 'Candle Color'] == 'red' or df.loc[i-1, 'Candle Color'] == 'doji') and df.loc[i-2, 'Candle Color'] == 'green' and df.loc[i-1, 'Low'] > df.loc[i-2, 'Low']:
            df.loc[i, 'A Engulfing Wave'] = 1
        # if red, then sell trade, look if got ABC pattern, B highest must smaller than A highest
        elif df.loc[i, 'Candle Color'] == 'red' and (df.loc[i-1, 'Candle Color'] == 'green' or df.loc[i-1, 'Candle Color'] == 'doji') and df.loc[i-2, 'Candle Color'] == 'red'and df.loc[i-1, 'High'] < df.loc[i-2, 'High']:
            df.loc[i, 'A Engulfing Wave'] = 1

In [ ]:
pattern = df[df['A Engulfing Wave'] == 1]
pattern

,Date,Time,Open,High,Low,Close,Volume,Candle Color,Candle Pattern,A Engulfing Wave
786,2023.10.13,14:00,1887.50,1891.99,1887.35,1891.25,3311,green,A engulfing,1
877,2023.10.16,13:45,1915.93,1918.35,1915.52,1917.98,1853,green,A engulfing,1
2206,2023.11.06,01:00,1992.70,1992.71,1988.73,1988.93,964,red,A engulfing,1
3084,2023.11.17,13:30,1990.95,1991.04,1987.33,1987.87,2479,red,A engulfing,1


In [ ]:
# Initialize the capital
capital = 10000

# Initialize the counters for winning trades, losing trades and cases where both levels are hit
winning_trades = 0
losing_trades = 0
both_levels_hit = 0

for i in range(0, len(df)):
    if df['A Engulfing Wave'].iloc[i] == 1:
      trade_open = True
      entry_price = df['Open'].iloc[i+1]
      # long the trade
      if df['Candle Color'].iloc[i] == 'green':
        stop_loss = entry_price - 5
        take_profit = entry_price + 5
        num_of_candle = i
        while trade_open == True:
          if df['Low'].iloc[num_of_candle+1] <= stop_loss and df['High'].iloc[num_of_candle+1] >= take_profit:
            both_levels_hit += 1
            trade_open = False
          else:
            if df['Low'].iloc[num_of_candle+1] <= stop_loss:
              capital *= 0.99
              losing_trades += 1
              trade_open = False
            elif df['High'].iloc[num_of_candle+1] >= take_profit:
              capital *= 1.01
              winning_trades += 1
              trade_open = False
          num_of_candle += 1
      # short the trade
      elif df['Candle Color'].iloc[i] == 'red':
        stop_loss = entry_price + 5
        take_profit = entry_price - 5
        num_of_candle = i
        while trade_open == True:
          if df['High'].iloc[num_of_candle+1] >= stop_loss and df['Low'].iloc[num_of_candle+1] <= take_profit:
            both_levels_hit += 1
            trade_open = False
          else:
            if df['High'].iloc[num_of_candle+1] >= stop_loss:
              capital *= 0.99
              losing_trades += 1
              trade_open = False
            elif df['Low'].iloc[num_of_candle+1] <= take_profit:
              capital *= 1.01
              winning_trades += 1
              trade_open = False
          num_of_candle += 1
      # no trade
      else:
        trade_open = False
        print('The engulfing candle is neither green or red!')


# Print the final capital and the number of winning and losing trades
total_trades = winning_trades + losing_trades
winning_rate = (winning_trades / total_trades) * 100

print(f"Final capital: {capital}")
print(f"Number of winning trades: {winning_trades}")
print(f"Number of losing trades: {losing_trades}")
print(f"Number of cases where both levels are hit: {both_levels_hit}")
print(f"Winning rate: {winning_rate:.2f}%")

Final capital: 10406.0401
Number of winning trades: 4
Number of losing trades: 0
Number of cases where both levels are hit: 0
Winning rate: 100.00%


##A Engulfing Wave (Gold 30)

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/sample.csv')
columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD15min3Oct2023.csv', names=columns, header=None)
df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD30min1Sept2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD1hour10July2023.csv', names=columns, header=None)
#df = pd.read_csv('/content/drive/MyDrive/交易成绩记录/dataset/mt4/XAUUSD4hour8July2022.csv', names=columns, header=None)
df

,Date,Time,Open,High,Low,Close,Volume
0,2023.09.01,02:30,1939.78,1940.09,1939.60,1939.87,322
1,2023.09.01,03:00,1939.93,1940.21,1938.89,1939.31,820
2,2023.09.01,03:30,1939.31,1939.44,1938.67,1939.32,813
3,2023.09.01,04:00,1939.51,1944.05,1939.28,1942.82,1911
4,2023.09.01,04:30,1942.83,1943.08,1942.21,1942.58,1675
...,...,...,...,...,...,...,...
2689,2023.11.22,15:00,2001.86,2003.17,2000.59,2000.59,7058
2690,2023.11.22,15:30,2000.23,2000.83,1996.83,1999.17,9302
2691,2023.11.22,16:00,1999.19,2000.81,1998.50,1999.55,8543
2692,2023.11.22,16:30,1999.56,2002.39,1998.82,2000.99,9375


In [ ]:
# Add 'Candle Color' column
df['Candle Color'] = np.where(df['Open'] > df['Close'] + 0.05, 'red', np.where(df['Open'] < df['Close'] - 0.05, 'green', 'doji'))

# Add 'Candle Pattern' column
df['Candle Pattern'] = 'other'

# 参数调整
# DOJI 烛体的大小
doji_body_threshold = 0.05
# HAMMER 烛体的长度比例
hammer_body_ratio_from = 5/18
hammer_body_ratio_to = 7/18
# HAMMER 烛体方影线的长度比例
hammer_shadow_ratio = 1/14


# DOJI
# The absolute length of the real body is less or equal to doji_body_threshold
doji_condition = (df['Open'] - df['Close']).abs() <= doji_body_threshold


# HAMMER
# - The absolute length of the real body must be:
#   - Greater than or equal to hammer_body_ratio_from of the entire candle range.
#   - Less than or equal to hammer_body_ratio_to of the entire candle range.
# - Also either:
#   - The upper shadow is less than or equal to hammer_shadow_ratio of the candle range.
#   - OR the lower shadow is less than or equal to hammer_shadow_ratio of the candle range.
hammer_condition = (((df['Open'] - df['Close']).abs() >= (df['High'] - df['Low']) * hammer_body_ratio_from) & \
                   ((df['Open'] - df['Close']).abs() <= (df['High'] - df['Low']) * hammer_body_ratio_to) & \
                   (((df['High'] - df[['Open', 'Close']].max(axis=1)) <= (df['High'] - df['Low']) * hammer_shadow_ratio) | \
                   ((df[['Open', 'Close']].min(axis=1) - df['Low']) <= (df['High'] - df['Low']) * hammer_shadow_ratio)))


# ENGULFING
# - B-Engulfing: The current candle body high is higher and the body low is lower than the previous candle.
# - A-Engulfing: The current candle body high is higher and the body low is lower than the previous candle shadow.
b_engulfing_condition = (df[['Open', 'Close']].max(axis=1) >= df[['Open', 'Close']].max(axis=1).shift(1)) & \
                        (df[['Open', 'Close']].min(axis=1) <= df[['Open', 'Close']].min(axis=1).shift(1))
a_engulfing_condition = (df[['Open', 'Close']].max(axis=1) >= df['High'].shift(1)) & \
                        (df[['Open', 'Close']].min(axis=1) <= df['Low'].shift(1))


# Apply hammer condition
df.loc[doji_condition, 'Candle Pattern'] = 'doji'
df.loc[hammer_condition, 'Candle Pattern'] = 'hammer'
df.loc[b_engulfing_condition, 'Candle Pattern'] = 'B engulfing'
df.loc[a_engulfing_condition, 'Candle Pattern'] = 'A engulfing'

In [ ]:
pattern = df[df['Candle Color'] == 'doji']
pattern

,Date,Time,Open,High,Low,Close,Volume,Candle Color,Candle Pattern
2,2023.09.01,03:30,1939.31,1939.44,1938.67,1939.32,813,doji,doji
20,2023.09.01,12:30,1944.21,1944.34,1943.79,1944.22,1157,doji,doji
48,2023.09.04,03:30,1940.91,1941.69,1940.76,1940.90,887,doji,doji
63,2023.09.04,11:00,1943.92,1944.46,1943.50,1943.91,1186,doji,doji
79,2023.09.04,19:00,1938.62,1938.86,1937.95,1938.57,710,doji,doji
...,...,...,...,...,...,...,...,...,...
2566,2023.11.17,22:30,1980.73,1980.96,1980.01,1980.77,2241,doji,doji
2571,2023.11.20,02:00,1974.56,1975.08,1973.28,1974.60,2538,doji,doji
2620,2023.11.21,03:30,1989.91,1990.32,1988.00,1989.95,4520,doji,doji
2656,2023.11.21,21:30,2000.04,2000.62,1999.22,2000.06,3831,doji,doji


In [ ]:
# define a new column 'A Engulfing Wave'
df['A Engulfing Wave'] = 0

# iterate over the DataFrame
for i in range(2, len(df)):
    # check if current 'Candle Pattern' is 'A Engulfing'
    if df.loc[i, 'Candle Pattern'] == 'A engulfing':
        # if green, then buy trade, look if got ABC pattern, B lowest must bigger than A lowest
        if df.loc[i, 'Candle Color'] == 'green' and (df.loc[i-1, 'Candle Color'] == 'red' or df.loc[i-1, 'Candle Color'] == 'doji') and df.loc[i-2, 'Candle Color'] == 'green' and df.loc[i-1, 'Low'] > df.loc[i-2, 'Low']:
            df.loc[i, 'A Engulfing Wave'] = 1
        # if red, then sell trade, look if got ABC pattern, B highest must smaller than A highest
        elif df.loc[i, 'Candle Color'] == 'red' and (df.loc[i-1, 'Candle Color'] == 'green' or df.loc[i-1, 'Candle Color'] == 'doji') and df.loc[i-2, 'Candle Color'] == 'red'and df.loc[i-1, 'High'] < df.loc[i-2, 'High']:
            df.loc[i, 'A Engulfing Wave'] = 1

In [ ]:
pattern = df[df['A Engulfing Wave'] == 1]
pattern

,Date,Time,Open,High,Low,Close,Volume,Candle Color,Candle Pattern,A Engulfing Wave
445,2023.09.14,20:30,1911.42,1911.42,1908.37,1908.70,1484,red,A engulfing,1
708,2023.09.22,13:30,1926.19,1926.42,1924.30,1924.68,1111,red,A engulfing,1
823,2023.09.27,02:00,1900.84,1901.64,1900.84,1901.44,345,green,A engulfing,1
2523,2023.11.17,01:00,1980.64,1981.82,1980.50,1981.65,477,green,A engulfing,1


In [ ]:
# Initialize the capital
capital = 10000

# Initialize the counters for winning trades, losing trades and cases where both levels are hit
winning_trades = 0
losing_trades = 0
both_levels_hit = 0

for i in range(0, len(df)):
    if df['A Engulfing Wave'].iloc[i] == 1:
      trade_open = True
      entry_price = df['Open'].iloc[i+1]
      # long the trade
      if df['Candle Color'].iloc[i] == 'green':
        stop_loss = entry_price - 5
        take_profit = entry_price + 5
        num_of_candle = i
        while trade_open == True:
          if df['Low'].iloc[num_of_candle+1] <= stop_loss and df['High'].iloc[num_of_candle+1] >= take_profit:
            both_levels_hit += 1
            trade_open = False
          else:
            if df['Low'].iloc[num_of_candle+1] <= stop_loss:
              capital *= 0.99
              losing_trades += 1
              trade_open = False
            elif df['High'].iloc[num_of_candle+1] >= take_profit:
              capital *= 1.01
              winning_trades += 1
              trade_open = False
          num_of_candle += 1
      # short the trade
      elif df['Candle Color'].iloc[i] == 'red':
        stop_loss = entry_price + 5
        take_profit = entry_price - 5
        num_of_candle = i
        while trade_open == True:
          if df['High'].iloc[num_of_candle+1] >= stop_loss and df['Low'].iloc[num_of_candle+1] <= take_profit:
            both_levels_hit += 1
            trade_open = False
          else:
            if df['High'].iloc[num_of_candle+1] >= stop_loss:
              capital *= 0.99
              losing_trades += 1
              trade_open = False
            elif df['Low'].iloc[num_of_candle+1] <= take_profit:
              capital *= 1.01
              winning_trades += 1
              trade_open = False
          num_of_candle += 1
      # no trade
      else:
        trade_open = False
        print('The engulfing candle is neither green or red!')


# Print the final capital and the number of winning and losing trades
total_trades = winning_trades + losing_trades
winning_rate = (winning_trades / total_trades) * 100

print(f"Final capital: {capital}")
print(f"Number of winning trades: {winning_trades}")
print(f"Number of losing trades: {losing_trades}")
print(f"Number of cases where both levels are hit: {both_levels_hit}")
print(f"Winning rate: {winning_rate:.2f}%")

Final capital: 9998.000100000001
Number of winning trades: 2
Number of losing trades: 2
Number of cases where both levels are hit: 0
Winning rate: 50.00%


##Support Resistance Lines

In [ ]:
from scipy.signal import argrelextrema
# Define the number of points to be checked around each peak and trough for support and resistance validation
n = 20

# Find local minima and maxima
local_min = df.iloc[argrelextrema(df['Low'].values, np.less_equal, order=n)]['Low']
local_max = df.iloc[argrelextrema(df['High'].values, np.greater_equal, order=n)]['High']

# Combine local minima and maxima into one Series
levels = pd.concat([local_min, local_max])

print("Support and Resistance levels are:\n", levels)

Support and Resistance levels are:
 16      1815.18
62      1819.30
124     1816.67
172     1816.62
240     1813.04
         ...   
3079    1993.37
3140    1985.13
3254    1994.28
3282    2007.50
3346    2006.37
Length: 114, dtype: float64
